In [1]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [2]:
# Vectorization imports
import re
import spacy
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords

## Data

In [2]:
df = pd.read_csv('../data/data_final.csv')
df.head(2)

,subject_id,gender,anchor_age,lengths_of_1st_admission,lengths_of_2nd_admission,0.45% sodium chloride Sum,1/2 ns Sum,allopurinol Sum,alprazolam Sum,amiodarone Sum,amlodipine Sum,aspirin Sum,aspirin ec Sum,benzonatate Sum,bisacodyl Sum,captopril Sum,carvedilol Sum,ciprofloxacin iv Sum,clonazepam Sum,d5 1/2ns Sum,d5ns Sum,diltiazem Sum,diltiazem extended-release Sum,famotidine Sum,fluoxetine Sum,furosemide Sum,glucagon Sum,hydrochlorothiazide Sum,ibuprofen Sum,influenza vaccine quadrivalent Sum,influenza virus vaccine Sum,isosorbide mononitrate (extended release) Sum,labetalol Sum,lamotrigine Sum,lidocaine 1% (for picc/midline insertions) Sum,lisinopril Sum,losartan potassium Sum,lr Sum,magnesium oxide Sum,metformin (glucophage) Sum,metoprolol succinate xl Sum,midazolam Sum,mirtazapine Sum,mycophenolate mofetil Sum,neomycin-polymyxin-bacitracin Sum,nitroglycerin sl Sum,olanzapine Sum,olanzapine (disintegrating tablet) Sum,ondansetron Sum,ondansetron odt Sum,oxycodone sr (oxycontin) Sum,pneumococcal 23-valent polysaccharide vaccine Sum,pneumococcal vac polyvalent Sum,potassium chl 20 meq / 1000 ml d5 1/2 ns Sum,potassium chloride (powder) Sum,potassium chloride replacement (critical care and oncology) Sum,potassium chloride replacement (oncology) Sum,pravastatin Sum,prochlorperazine Sum,quetiapine fumarate Sum,ramelteon Sum,ranitidine Sum,rosuvastatin calcium Sum,sodium chloride 0.9% flush Sum,soln Sum,sterile water Sum,sw Sum,tamsulosin Sum,torsemide Sum,tramadol Sum,trazodone Sum,valsartan Sum,zolpidem tartrate Sum,0.45% sodium chloride Average,1/2 ns Average,allopurinol Average,alprazolam Average,amiodarone Average,amlodipine Average,aspirin Average,aspirin ec Average,benzonatate Average,bisacodyl Average,captopril Average,carvedilol Average,ciprofloxacin iv Average,clonazepam Average,d5 1/2ns Average,d5ns Average,diltiazem Average,diltiazem extended-release Average,famotidine Average,fluoxetine Average,furosemide Average,glucagon Average,hydrochlorothiazide Average,ibuprofen Average,influenza vaccine quadrivalent Average,influenza virus vaccine Average,isosorbide mononitrate (extended release) Average,labetalol Average,lamotrigine Average,lidocaine 1% (for picc/midline insertions) Average,lisinopril Average,losartan potassium Average,lr Average,magnesium oxide Average,metformin (glucophage) Average,metoprolol succinate xl Average,midazolam Average,mirtazapine Average,mycophenolate mofetil Average,neomycin-polymyxin-bacitracin Average,nitroglycerin sl Average,olanzapine Average,olanzapine (disintegrating tablet) Average,ondansetron Average,ondansetron odt Average,oxycodone sr (oxycontin) Average,pneumococcal 23-valent polysaccharide vaccine Average,pneumococcal vac polyvalent Average,potassium chl 20 meq / 1000 ml d5 1/2 ns Average,potassium chloride (powder) Average,potassium chloride replacement (critical care and oncology) Average,potassium chloride replacement (oncology) Average,pravastatin Average,prochlorperazine Average,quetiapine fumarate Average,ramelteon Average,ranitidine Average,rosuvastatin calcium Average,sodium chloride 0.9% flush Average,soln Average,sterile water Average,sw Average,tamsulosin Average,torsemide Average,tramadol Average,trazodone Average,valsartan Average,zolpidem tartrate Average,"Acute kidney failure, unspecified",Acute posthemorrhagic anemia,"Alcohol abuse, unspecified","Anemia, unspecified","Anxiety disorder, unspecified","Anxiety state, unspecified",Aortocoronary bypass status,"Asthma, unspecified type, unspecified",Atherosclerotic heart disease of native coronary artery without angina pectoris,Atrial fibrillation,"Chronic airway obstruction, not elsewhere classified","Chronic kidney disease, unspecified","Chronic obstructive pulmonary disease, unspecified","Congestive heart failure, unspecified",Coronary atherosclerosis of native coronary artery,"Coronary atherosclerosis of unspecified type of vessel, native or graft",Dehydration,"Depressive disorder, not elsewhere classified","Diabetes me

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31386 entries, 0 to 31385
Columns: 216 entries, subject_id to patientweight
dtypes: float64(139), int64(75), object(2)
memory usage: 51.7+ MB


In [11]:
df.shape

(31386, 216)

In [12]:
df.subject_id.nunique()

31386

In [13]:
df.describe()

,subject_id,anchor_age,lengths_of_1st_admission,lengths_of_2nd_admission,0.45% sodium chloride Sum,1/2 ns Sum,allopurinol Sum,alprazolam Sum,amiodarone Sum,amlodipine Sum,aspirin Sum,aspirin ec Sum,benzonatate Sum,bisacodyl Sum,captopril Sum,carvedilol Sum,ciprofloxacin iv Sum,clonazepam Sum,d5 1/2ns Sum,d5ns Sum,diltiazem Sum,diltiazem extended-release Sum,famotidine Sum,fluoxetine Sum,furosemide Sum,glucagon Sum,hydrochlorothiazide Sum,ibuprofen Sum,influenza vaccine quadrivalent Sum,influenza virus vaccine Sum,isosorbide mononitrate (extended release) Sum,labetalol Sum,lamotrigine Sum,lidocaine 1% (for picc/midline insertions) Sum,lisinopril Sum,losartan potassium Sum,lr Sum,magnesium oxide Sum,metformin (glucophage) Sum,metoprolol succinate xl Sum,midazolam Sum,mirtazapine Sum,mycophenolate mofetil Sum,neomycin-polymyxin-bacitracin Sum,nitroglycerin sl Sum,olanzapine Sum,olanzapine (disintegrating tablet) Sum,ondansetron Sum,ondansetron odt Sum,oxycodone sr (oxycontin) Sum,pneumococcal 23-valent polysaccharide vaccine Sum,pneumococcal vac polyvalent Sum,potassium chl 20 meq / 1000 ml d5 1/2 ns Sum,potassium chloride (powder) Sum,potassium chloride replacement (critical care and oncology) Sum,potassium chloride replacement (oncology) Sum,pravastatin Sum,prochlorperazine Sum,quetiapine fumarate Sum,ramelteon Sum,ranitidine Sum,rosuvastatin calcium Sum,sodium chloride 0.9% flush Sum,soln Sum,sterile water Sum,sw Sum,tamsulosin Sum,torsemide Sum,tramadol Sum,trazodone Sum,valsartan Sum,zolpidem tartrate Sum,0.45% sodium chloride Average,1/2 ns Average,allopurinol Average,alprazolam Average,amiodarone Average,amlodipine Average,aspirin Average,aspirin ec Average,benzonatate Average,bisacodyl Average,captopril Average,carvedilol Average,ciprofloxacin iv Average,clonazepam Average,d5 1/2ns Average,d5ns Average,diltiazem Average,diltiazem extended-release Average,famotidine Average,fluoxetine Average,furosemide Average,glucagon Average,hydrochlorothiazide Average,ibuprofen Average,influenza vaccine quadrivalent Average,influenza virus vaccine Average,isosorbide mononitrate (extended release) Average,labetalol Average,lamotrigine Average,lidocaine 1% (for picc/midline insertions) Average,lisinopril Average,losartan potassium Average,lr Average,magnesium oxide Average,metformin (glucophage) Average,metoprolol succinate xl Average,midazolam Average,mirtazapine Average,mycophenolate mofetil Average,neomycin-polymyxin-bacitracin Average,nitroglycerin sl Average,olanzapine Average,olanzapine (disintegrating tablet) Average,ondansetron Average,ondansetron odt Average,oxycodone sr (oxycontin) Average,pneumococcal 23-valent polysaccharide vaccine Average,pneumococcal vac polyvalent Average,potassium chl 20 meq / 1000 ml d5 1/2 ns Average,potassium chloride (powder) Average,potassium chloride replacement (critical care and oncology) Average,potassium chloride replacement (oncology) Average,pravastatin Average,prochlorperazine Average,quetiapine fumarate Average,ramelteon Average,ranitidine Average,rosuvastatin calcium Average,sodium chloride 0.9% flush Average,soln Average,sterile water Average,sw Average,tamsulosin Average,torsemide Average,tramadol Average,trazodone Average,valsartan Average,zolpidem tartrate Average,"Acute kidney failure, unspecified",Acute posthemorrhagic anemia,"Alcohol abuse, unspecified","Anemia, unspecified","Anxiety disorder, unspecified","Anxiety state, unspecified",Aortocoronary bypass status,"Asthma, unspecified type, unspecified",Atherosclerotic heart disease of native coronary artery without angina pectoris,Atrial fibrillation,"Chronic airway obstruction, not elsewhere classified","Chronic kidney disease, unspecified","Chronic obstructive pulmonary disease, unspecified","Congestive heart failure, unspecified",Coronary atherosclerosis of native coronary artery,"Coronary atherosclerosis of unspecified type of vessel, native or graft",Dehydration,"Depressive disorder, not elsewhere classified","Diabetes mellitus 

In [14]:
df.isnull().sum()

subject_id                        0
gender                            0
anchor_age                        0
lengths_of_1st_admission          0
lengths_of_2nd_admission          0
                               ... 
SURGICAL SAME DAY ADMISSION       0
URGENT                            0
sum_drg_severity                  0
sum_drg_mortality                 0
patientweight                  4413
Length: 216, dtype: int64

In [15]:
df = df.dropna().reset_index(drop=True)

In [16]:
df.drop_duplicates()
df.shape

(20658, 216)

### Encode columns

In [17]:
# Convert gender to binary
df['gender'] = df['gender'].map({'F': 0, 'M': 1})

### Vectorizing the Discharge Note

In [18]:
# Displaying the content of one discharge note in order to understand the structure of the data
cell_content = df["text"][1]
print(cell_content)

 
Name:  ___                 Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
omeprazole
 
Attending: ___.
 
Chief Complaint:
dysphagia 
 
Major Surgical or Invasive Procedure:
Upper endoscopy ___

 
History of Present Illness:
 ___ w/ anxiety and several years of dysphagia who p/w worsened 
foreign body sensation. 

She describes feeling as though food gets stuck in her neck when 
she eats. She put herself on a pureed diet to address this over 
the last 10 days. When she has food stuck in the throat, she 
almost feels as though she cannot breath, but she denies trouble 
breathing at any other time. She does not have any history of 
food allergies or skin rashes. 
 
In the ED, initial vitals:  97.6  81  148/83  16  100% RA 
Imaging showed: CXR showed a prominent esophagus
Consults: GI was consulted.

Pt underwent EGD which showed a normal appearing esophagus. 
Biopsies were taken.

Curr

#### Text Cleaning Function

In [24]:
# Load Spacy model
nlp = spacy.load('en_core_web_sm')

In [25]:
# Create our stopwords list including medical stopwords and ensuring negations stay
stop_words = set(stopwords.words("english"))
medical_stopwords = {"patient", "history", "medications", "symptoms", "treatment", "diagnosis", "discharge","date","birth","sex"}
stop_words.update(medical_stopwords)
negations = {"no", "not", "never", "none"}
stop_words -= negations  # Ensure negations are not removed


In [26]:
# Create our cleaning function
def preprocess_text(text):
    if isinstance(text, str):
        
        text = re.sub(r"(Name:|Unit No:|Admission Date:|Discharge Date:).*?\n", " ", text, flags=re.IGNORECASE | re.DOTALL) # Remove administrative headers (Name, Admission Date, Discharge Date)
        text = re.sub(r"\b(?:no|not|never|none)\b\s+(\w+(?:\s+\w+){0,2})", r"\1_not", text)  # Handle negations
        text = re.sub(r"[^a-zA-Z0-9\s]", " ", text) # Remove special characters but keep numbers (for vitals, dosages, etc.)
        text = re.sub(r"\s+", " ", text).strip() # Remove extra spaces
        text = text.lower()
        # Tokenization & Lemmatization
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if token.text not in stop_words]

        return tokens
    return []

In [27]:
# Apply the cleaning function to the discharge notes and creating a new row
df['cleaned_discharge_summary'] = df['text'].apply(preprocess_text)

#### Tokenizing and Vectorizing

In [28]:
# Create a list conraining tokenized words
documents = [TaggedDocument(words=row, tags=[str(index)]) for index, row in enumerate(df['cleaned_discharge_summary'])]

In [29]:
# Initiate the model that generates the vectors
model = Doc2Vec(documents, vector_size=300, window=10, min_count=1, workers=4, epochs=100, dm=1)

In [31]:
# Generate vector
df['doc2vec_vector'] = df['cleaned_discharge_summary'].apply(lambda x: model.infer_vector(x,alpha=0.1,epochs=20))

In [35]:
# Expand Vectors into dataframe
doc2vec_array = np.vstack(df["doc2vec_vector"].values) 

# Convert the array into a DataFrame with column names
doc2vec_df = pd.DataFrame(doc2vec_array, columns=[f"doc2vec_{i}" for i in range(doc2vec_array.shape[1])])

# Ensure the new DataFrame has the same index as the original
doc2vec_df.index = df.index

# Merge everything at once 
df = pd.concat([df, doc2vec_df], axis=1)

In [36]:
# Drop Unnecessary Columns
df.drop(columns=["text", "doc2vec_vector"], inplace=True)


In [37]:
# Save Processed Data
df.to_csv("../data/dataset_with_vec.csv", index=False)